In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

intakes_file_path = os.path.join("center_data", "Austin_Animal_Center_Intakes.csv")

dog_intakes_df = pd.read_csv(intakes_file_path)\
    .rename(columns = lambda df: df.lower().replace(" ", "_"))\
    .drop("monthyear", axis=1)

dog_intakes_df = dog_intakes_df[dog_intakes_df["animal_type"] == 'Dog']\
    .rename(columns = {
    "intake_type":"type",
    "sex_upon_intake":"sex",
    "age_upon_intake":"age"
    })\
    .assign(is_intake=True)\
    .assign(datetime = lambda df: pd.to_datetime(df["datetime"]))\
    .reset_index(drop=True)

# dogs_intakes_df.to_csv('dog_intakes.csv')
dog_intakes_df.head()

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake
0,A786884,*Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,True
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,True
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,True
3,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,True
4,A743852,Odin,2017-02-18 12:46:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,2 years,Labrador Retriever Mix,Chocolate,True


In [56]:
outcomes_file_path = os.path.join("center_data", "Austin_Animal_Center_Outcomes.csv")

dog_outcomes_df = pd.read_csv(outcomes_file_path)\
    .rename(columns = lambda df: df.lower().replace(" ", "_"))\
    .drop("monthyear", axis=1)

dog_outcomes_df = dog_outcomes_df[
    (dog_outcomes_df["animal_type"] == 'Dog') & (dog_outcomes_df['outcome_type'] == 'Adoption')]\
    .rename(columns = {
        "outcome_type":"type",
        "sex_upon_outcome":"sex",
        "age_upon_outcome":"age"
        })\
    .assign(is_intake=False)\
    .assign(datetime = lambda df: pd.to_datetime(df["datetime"]))\
    .reset_index(drop=True)    

# dogs_intakes_df.to_csv('dog_intakes.csv')
dog_outcomes_df.head(5)

,animal_id,name,datetime,date_of_birth,type,outcome_subtype,animal_type,sex,age,breed,color,is_intake
0,A789027,Lennie,2019-02-17 11:44:00,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream,False
1,A720371,Moose,2016-02-13 17:59:00,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,False
2,A789027,Lennie,2019-03-10 12:25:00,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream,False
3,A794494,Zoey,2019-05-14 16:27:00,06/14/2018,Adoption,NaN,Dog,Spayed Female,10 months,Wire Hair Fox Terrier Mix,White/Black,False
4,A764361,Astro,2018-01-03 19:15:00,12/28/2016,Adoption,NaN,Dog,Neutered Male,1 year,Norwich Terrier Mix,Black/Tan,False


In [206]:
dog_concat_df = pd\
    .concat([dog_intakes_df, dog_outcomes_df], sort=False)\
    .sort_values(by=["animal_id", "datetime"])\
     [lambda df: df.duplicated(["animal_id"],keep=False)]\
    .fillna("N/A")
# dog_concat_df[dog_concat_df["animal_id"] == "A047759"]


dog_concat_df.head()

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake,date_of_birth,outcome_subtype
39209,A006100,Scamp,2014-03-07 14:26:00,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
1859,A006100,Scamp,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
8663,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
45817,A200922,Carlos,2013-10-03 15:47:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,True,N/A,N/A
5096,A200922,Carlos,2013-11-22 09:44:00,N/A,Adoption,N/A,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,False,10/03/1997,Foster


In [207]:
describe_df = dog_concat_df.describe()
describe_df

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake,date_of_birth,outcome_subtype
count,68444,68444,68444,68444,68444,68444,68444,68444,68444,68444,68444,68444,68444,68444
unique,28509,8561,60807,20320,5,11,1,6,47,1696,294,2,4677,3
top,A721033,N/A,2014-02-19 13:51:00,N/A,Adoption,Normal,Dog,Neutered Male,1 year,Pit Bull Mix,Black/White,True,N/A,N/A
freq,33,10827,47,29906,29906,36504,68444,23004,14926,8118,8605,38538,38538,64500
first,NaN,NaN,2013-10-01 11:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2020-01-07 17:18:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [208]:
# dog_concat_df.groupby(dog_concat_df["animal_id"], as_index=False).count()
num_records_df = dog_concat_df["animal_id"]\
    .value_counts()\
    .to_frame()\
    .reset_index()\
    .rename(columns = {
        "index":"animal_id",
        "animal_id":"num_records"
    })

num_records_df

,animal_id,num_records
0,A721033,33
1,A754989,16
2,A770009,14
3,A718223,14
4,A737854,13
...,...,...
28504,A777878,2
28505,A795219,2
28506,A651386,2
28507,A779017,2


In [46]:
boomerang_df = pd.DataFrame(columns = ["animal_id", "num_records"])

def is_odd(num):
    if num%2 == 0:
        return False
    else:
        return True

def greater_than_one(num):
    if num>1:
        return True
    else:
        return False

boomerang_df = num_records_df[num_records_df["num_records"].map(greater_than_one)]
boomerang_df = boomerang_df[boomerang_df["num_records"].map(is_odd)]

boomerang_df_full = dog_concat_df.loc[dog_concat_df["animal_id"].isin(boomerang_df["animal_id"])]
boomerang_df_full

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake,date_of_birth,outcome_subtype
39209,A006100,Scamp,2014-03-07 14:26:00,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White,True,NaN,NaN
1859,A006100,Scamp,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White,True,NaN,NaN
8663,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,True,NaN,NaN
30441,A338231,Boots,2015-05-18 14:14:00,Bullard/Northland in Austin (TX),Stray,Normal,Dog,Neutered Male,13 years,Border Collie Mix,Black/White,True,NaN,NaN
40501,A338231,Boots,2018-02-25 14:30:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,16 years,Border Collie Mix,Black/White,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12263,A811071,Bailey,2019-12-30 11:37:00,NaN,Adoption,NaN,Dog,Spayed Female,2 years,Black Mouth Cur Mix,Tan,False,12/27/2017,NaN
60363,A811071,Bailey,2020-01-03 11:35:00,Outside Jurisdiction,Owner Surrender,Normal,Dog,Spayed Female,2 years,Black Mouth Cur Mix,Tan,True,NaN,NaN
60438,A811116,Koda,2019-12-28 12:11:00,9511 Burklund Farms Road in Travis (TX),Stray,Normal,Dog,Intact Male,5 years,Siberian Husky/Akita,Gray/White,True,NaN,NaN
12464,A811116,Koda,2020-01-02 16:22:00,NaN,Adoption,NaN,Dog,Neutered Male,5 years,Siberian Husky/Akita,Gray/White,False,12/28/2014,NaN


In [209]:
def qualified(num_intakes, num_outcomes):
    is_qualified = False
    
    if num_intakes == num_outcomes or num_intakes-1 == num_outcomes:
        is_qualified=True
    
    return is_qualified


#     elif is_odd(num_records):
#         if num_records//2+1 == num_intakes:
#             is_qualified = True
#     else:
#         if num_records//2 == num_intakes:
#             is_qualified = True    
    

In [210]:
dog_intake_count_df = dog_concat_df[["animal_id", "is_intake"]]\
    .groupby(by = ["animal_id"], as_index=False).count()

# intake_count_df = dog_qualified_df[dog_qualified_df["is_intake"] == True]["is_intake"]\


dog_intake_count_df
# intake_count_df

,animal_id,is_intake
0,A006100,3
1,A200922,2
2,A210457,2
3,A226069,2
4,A245945,2
...,...,...
28504,A811450,2
28505,A811507,2
28506,A811509,2
28507,A811534,2


In [211]:
intakes_agg = dog_concat_df[dog_concat_df["is_intake"]].groupby("animal_id")["animal_id"].count()
outcomes_agg = dog_concat_df[~dog_concat_df["is_intake"]].groupby("animal_id")["animal_id"].count()

In [212]:
validation_agg = pd.merge(
    intakes_agg.to_frame().rename(columns = {"animal_id":"intake_count"}),
    outcomes_agg.to_frame().rename(columns = {"animal_id":"outcome_count"}),
    how="outer",
    left_index=True,
    right_index=True
).dropna().reset_index()

# validation_agg

validation_agg["valid"] = np.vectorize(qualified)(validation_agg["intake_count"], validation_agg["outcome_count"])
matched_stays_df = validation_agg[validation_agg["valid"]]["animal_id"]
matched_stays_df

0        A200922
1        A210457
2        A226069
3        A249087
4        A274546
          ...   
26226    A811450
26227    A811507
26228    A811509
26229    A811534
26230    A811598
Name: animal_id, Length: 25802, dtype: object

In [214]:
working_df = dog_concat_df.loc[dog_concat_df["animal_id"].isin(matched_stays_df.values)]
working_df

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake,date_of_birth,outcome_subtype
45817,A200922,Carlos,2013-10-03 15:47:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,True,N/A,N/A
5096,A200922,Carlos,2013-11-22 09:44:00,N/A,Adoption,N/A,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,False,10/03/1997,Foster
35721,A210457,Caleb,2016-09-28 12:05:00,4424 S Mopac #412 in Austin (TX),Public Assist,Aged,Dog,Neutered Male,17 years,Chihuahua Shorthair,Tan/Black,True,N/A,N/A
9001,A210457,Caleb,2016-10-07 12:34:00,N/A,Adoption,N/A,Dog,Neutered Male,17 years,Chihuahua Shorthair,Tan/Black,False,06/01/1999,Foster
48812,A226069,Cedar,2015-10-06 12:29:00,Wheless Ln And Berkman Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,15 years,Labrador Retriever/Beagle,Sable/White,True,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12564,A811509,Bruno,2020-01-05 15:29:00,N/A,Adoption,N/A,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,Chocolate,False,04/01/2018,N/A
61861,A811534,Lilu,2020-01-05 13:01:00,Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,11 years,Dachshund,Black/Brown,True,N/A,N/A
12676,A811534,Lilu,2020-01-07 15:01:00,N/A,Adoption,N/A,Dog,Spayed Female,11 years,Dachshund,Black/Brown,False,01/05/2009,N/A
63060,A811598,Ginger,2020-01-06 12:09:00,Travis (TX),Owner Surrender,Normal,Dog,Spayed Female,8 years,Chihuahua Shorthair,White/Red,True,N/A,N/A


In [130]:
comparison_df = pd.merge(num_records_df, dog_intake_count_df, on="animal_id")
comparison_df

# qualified_df = qualified(num_records_df["num_records"], dog_intake_count_df["is_intake"])

,animal_id,num_records,is_intake
0,A721033,33,33
1,A754989,16,16
2,A770009,14,14
3,A718223,14,14
4,A737854,13,13
...,...,...,...
28504,A777878,2,2
28505,A795219,2,2
28506,A651386,2,2
28507,A779017,2,2


In [ ]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata
import requests

app_token="3mubvsyw1rybaisquwqluss7zqf6c3nqxu5g4x56odov7ggexx"
def get_soda_api_data(endpoint, app_token, count=1000, offset=0, return_df=True):
    params = {'$$app_token': app_token, '$limit': count, '$offset': offset, }
    
    results = []

    while True:

        try:
            r = requests.get(endpoint, params=params)
            rcontent = r.json()

            if rcontent == []:
                break

            results.append(rcontent)
            offset += count 
            params['$offset'] = offset

        except HTTPError as err:

            if err.response.status_code == '404':
                break
            else:
                print(err.response.status_code)
    
    if return_df:
        results_df = pd.DataFrame()

        for i in results:
            results_df = results_df.append(pd.io.json.json_normalize(i))
        
        return results_df
    
    else:
        return results
    
endpoint="https://data.austintexas.gov/resource/wter-evkm.json"

get_soda_api_data(endpoint, app_token)